In [63]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [64]:
headers = {'Accept-Language': 'en-US,en;q=0.8'}
url = 'https://ed.ted.com/lessons?direction=desc&sort=featured-position&user_by_click=student'
response = requests.get(url,headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
print(soup)

<!DOCTYPE html>

<html class="no-js" lang="en">
<head>
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-MCGS9HPEJD"></script>
<script>
      window.dataLayer = window.dataLayer || [];
      function gtag(){
        dataLayer.push(arguments);
      }
      gtag('js', new Date());
      if (window.location.hostname !== 'localhost' && window.location.hostname !== 'teded.test') {
        gtag('set', 'cookie_flags', 'SameSite=None;Secure');
        gtag('set', 'cookie_domain', 'ed.ted.com');
      }
      gtag('set', {
        'user_id': '',
        'user_properties': {
          'user_type': '',
          'user_by_click': 'student',
          'external_id': ''
        }
      });
      let event_params = {
        'auth_state': 'loggedOut'
      };
      if (document.querySelector('meta[name="lesson-id"]')) {
        event_params.lesson_id = document.querySelector('meta[name="lesson-id"]').getAttribute('content');
        event_params.lesson_official = document.querySele

In [65]:
lesson_links = ["https://ed.ted.com/" + l['href'] for l in soup.select('h2 a')]
print(lesson_links[0])

https://ed.ted.com//lessons/should-we-get-rid-of-standardized-testing-arlo-kempf


In [66]:

lesson_title = [a.get_text() for a in soup.select('h2 a')]
print(lesson_title)

['Should we get rid of standardized testing?', "How schools can nurture every student's genius - Trish Millines Dziko", "Why perfect grades don't matter", 'Why are there so many types of apples?', 'Why we should draw more (and photograph less)', 'Why should you read Flannery O’Connor?', 'What is dyslexia?', '3 rules to spark learning - Ramsey Musallam', 'How can we support the emotional well-being of teachers? - Sydney Jensen', 'Myths you learned in health class', 'The dark history of IQ tests', '5 tips to improve your critical thinking', 'What’s the smartest age?', "A Parkland teacher's homework for us all - Diane Wolk-Rogers", 'How (and why) to read William Faulkner', 'How well do masks work?', 'How playing sports benefits your body ... and your brain', "The benefits of a good night's sleep", 'Am I Really A Visual Learner?', 'How to teach kids to talk about taboo topics - Liz Kleinrock', 'How to design a library that makes kids want to read - Michael Bierut', 'The benefits of good po

In [67]:
lesson_category = [c.get_text() for c in soup.select('a.text-secondary-700')]
print(lesson_category)

['Education Policy', 'Teaching & Education', 'Teaching & Education', 'Science & Technology', 'Teaching & Education', 'Literature & Language', 'Medical Conditions', 'Teaching & Education', 'Teaching & Education', 'Health', 'History', 'Thinking & Learning', 'Thinking & Learning', 'Public Health', 'Literature & Language', 'Public Health', 'Physical Fitness', 'Health', 'Thinking & Learning', 'Teaching & Education', 'Design, Engineering & Technology', 'Health', 'Health', 'Numbers & Operations']


In [68]:
lesson_duration = [
    ' '.join(d.get_text().replace("Lesson duration ", "").split())
    for d in soup.select('p.sr-only')
]
print(lesson_duration)


['05:41', '15:41', '04:58', '04:28', '02:53', '04:12', '04:35', '06:30', '11:32', '05:03', '06:10', '04:30', '04:53', '15:47', '04:41', '08:21', '03:47', '05:45', '02:38', '12:02', '12:27', '04:27', '02:43', '04:30']


In [69]:
lesson_viewers = [
    ' '.join(v.get_text().replace(" Views", "").split())
    for v in soup.select('p.text-gray-700')
]
print(lesson_viewers)


['1,138,584', '68,571', '4,331,864', '1,024,416', '366,693', '365,147', '6,330,076', '375,106', '144,501', '516,284', '2,134,769', '9,714,935', '1,181,513', '66,820', '166,532', '2,142,410', '2,038,590', '6,201,380', '294,810', '136,987', '449,151', '13,338,929', '1,823,382', '2,298,774']


In [70]:
lesson_description = []

for link in lesson_links:
    response_link = requests.get(link, headers=headers)
    soup_link = BeautifulSoup(response_link.text, "html.parser")
    description = soup_link.select('.lessonDescription')
    cleaned_description = ' '.join(description[0].get_text().split()) if description else None
    lesson_description.append(cleaned_description)

print(lesson_description)

['Although standardized testing is a particularly hot topic in education right now, this approach to measurement has been in use for two millennia. And while the results of standardized testing can help us understand some things, they can also be misleading if used incorrectly. So what do these tests actually measure? And are they worthwhile? Arlo Kempf investigates.', 'Forget home economics and standardized tests, education visionary Trish Millines Dziko has a much more engaging and fulfilling way for students to develop real-world skills. Get schooled by Dziko as she shares how project-based learning can transform public education and unlock genius for the next generation of critical thinkers, problem solvers, ideators and leaders.', 'Most American students strive for a 4.0 GPA and the highest test scores, but research shows that this quest for perfection actually discourages creativity and reduces academic risk-taking. In this episode of “School Myths” by The Atlantic, Alice Roth in

In [72]:
df = pd.DataFrame(
    {'Title': lesson_title,'Category' : lesson_category,'Duration': lesson_duration,'Viewers': lesson_viewers,'Description': lesson_description,'Link': lesson_links})
print(df.head())
df.to_csv('teded.csv', index=False)

                                               Title              Category  \
0         Should we get rid of standardized testing?      Education Policy   
1  How schools can nurture every student's genius...  Teaching & Education   
2                    Why perfect grades don't matter  Teaching & Education   
3             Why are there so many types of apples?  Science & Technology   
4      Why we should draw more (and photograph less)  Teaching & Education   

  Duration    Viewers                                        Description  \
0    05:41  1,138,584  Although standardized testing is a particularl...   
1    15:41     68,571  Forget home economics and standardized tests, ...   
2    04:58  4,331,864  Most American students strive for a 4.0 GPA an...   
3    04:28  1,024,416  Have you ever walked into a grocery store and ...   
4    02:53    366,693  A photograph is a much better representation o...   

                                                Link  
0  https://ed.ted.c